# Loading and modifying layers and layer collections

In [ ]:
from datetime import datetime

import geoengine as ge

In [ ]:
ge.initialize("http://localhost:3030/api", credentials=("admin@localhost", "adminadmin"))

## Retrieve layer collections

In [ ]:
root_collection = ge.layer_collection()
root_collection

In [ ]:
root_of_layerdb = root_collection.get_items_by_name_unique(name="Data Catalog").load()
root_of_layerdb

In [ ]:
test_collection = root_of_layerdb.get_items_by_name_unique(name="A test collection").load()
test_collection

In [ ]:
layer = test_collection.get_items_by_name_unique(name="Ports in Germany").load()
layer

In [ ]:
workflow = layer.as_workflow()

time = datetime.strptime("2014-04-01T12:00:00.000Z", "%Y-%m-%dT%H:%M:%S.%f%z")

data = workflow.get_dataframe(
    ge.QueryRectangle(
        ge.BoundingBox2D(-111.533203125, -4.482421875, 114.345703125, 73.388671875),
        ge.TimeInterval(time, time)
    )
)

data

## Add collections and layers

In [ ]:
root_of_layerdb.add_collection("my test collection", "test description")
root_of_layerdb = root_of_layerdb.reload()
root_of_layerdb

In [ ]:
test_collection = root_of_layerdb.get_items_by_name_unique("my test collection").load()
test_collection

In [ ]:
test_collection.add_collection("sub collection", "another description")

test_collection.add_layer(
    name="ports clone",
    description="test description",
    workflow={
        "type": "Vector",
        "operator": {
            "type": "PointInPolygonFilter",
            "params": {},
            "sources": {
                "points": {
                    "type": "OgrSource",
                    "params": {"data": "ne_10m_ports", "attributeProjection": None, "attributeFilters": None},
                },
                "polygons": {
                    "type": "OgrSource",
                    "params": {"data": "germany_outline", "attributeProjection": None, "attributeFilters": None},
                },
            },
        },
    },
    symbology=None,
)

test_collection = test_collection.reload()
test_collection

In [ ]:
sub_collection = test_collection.items[0].load()
sub_collection.add_existing_layer(test_collection.items[1])

sub_collection.add_collection("sub sub collection", "yet another description")

sub_collection = sub_collection.reload()
sub_collection

In [ ]:
test_collection.add_existing_collection(sub_collection.items[0])

test_collection = test_collection.reload()
test_collection

## Delete previously added collections and layers

In [ ]:
test_collection

In [ ]:
test_collection.remove_item(0)
test_collection

In [ ]:
test_collection.remove_item(0)
test_collection

In [ ]:
test_collection.remove_item(0)
test_collection

In [ ]:
root_of_layerdb.reload()

In [ ]:
test_collection.remove()

In [ ]:
root_of_layerdb = root_of_layerdb.reload()
root_of_layerdb

## Search for layer in layer collection

In [ ]:
search_result = root_of_layerdb.search("ports")

print("Found…\n")
print(*search_result)

search_result[0].load()

In [ ]:
root_of_layerdb.search("ports")[0].load()